In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [2]:
from deeprte.experiment import Trainer

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from deeprte.config import make_config
config = make_config()

In [4]:
e = Trainer(mode = "train", init_rng=0, config = config)

In [5]:
import tensorflow as tf
tf.nest.map_structure(lambda x: x.shape, (e.tf_data))

{'sigma': TensorShape([8, 1681, 2]),
 'psi_label': TensorShape([8, 40344]),
 'scattering_kernel': TensorShape([8, 40344, 24]),
 'boundary': TensorShape([8, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}

In [6]:
train_input = e._build_train_input()

In [8]:
tf.nest.map_structure(lambda x: x.shape, next(train_input))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


{'sigma': (1, 6, 1681, 2),
 'psi_label': (1, 6, 500),
 'scattering_kernel': (1, 6, 500, 24),
 'boundary': (1, 6, 1968),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 500, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [11]:
val_input = e._build_val_input()

In [12]:
tf.nest.map_structure(lambda x: x.shape, next(val_input))

{'sigma': (1, 2, 1681, 2),
 'psi_label': (1, 2, 40344),
 'scattering_kernel': (1, 2, 40344, 24),
 'boundary': (1, 2, 1968),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 40344, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [13]:
ds = e._build_dummy_input()

AttributeError: 'Trainer' object has no attribute '_load_dummy_data'

In [ ]:
tf.nest.map_structure(lambda x: x.shape, ds)

{'boundary': (1, 1968),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'phase_coords': (1, 4),
 'position_coords': (1, 1681, 2),
 'psi_label': (1,),
 'scattering_kernel': (1, 24),
 'sigma': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'velocity_weights': (1, 24)}